In [1]:
import numpy as np
import itertools
import pprint 
pp= pprint.PrettyPrinter(indent=4)

In [2]:
T=10
times= np.arange(T)
def ho_lee_rn(n, heads, r0, lam, sigma):
    #definition of Ho-Lee model
    sig_coeff= (heads-(n-heads))
    return r0 + lam*n + sig_coeff*sigma

def get_all_rn(times, r0, lam, sigma):
    #compute interest rate tree here
    res= {}
    for i in times:
        res[i]= ho_lee_rn(i, np.arange(i+1), r0, lam, sigma)
    return res

all_rn= get_all_rn(np.arange(T), 0.06, 0, 0.004)

In [3]:
def payment(all_rn, curr_t, k, K=.062, F=1000):
    #definition of the payment function
    prev_r= all_rn[curr_t-1][k]
    return F*(K-prev_r)

def all_pay(all_rn, times):
    res= {}
    for t in times:
        q_t= np.empty(t)
        num_heads= np.arange(t)
        for k in num_heads:
            q_t[k]= payment(all_rn, t, k)
        res[t]= q_t
    return res

#payments at time 1 to 10
all_payments= all_pay(all_rn, np.arange(1, 11))

#q4: Receiver swap and swaption
def backward_swap(K=0.062, F=1000):
    terminal_val= all_payments[T]
    prices= {T-1: terminal_val/(1+all_rn[T-1])}
    remain_time= np.arange(T-2, -1, -1)
    for t in remain_time:
        tprices= np.empty(t+1)
        num_heads= np.arange(t+1)
        for k in num_heads:
            tprices[k]= 1/(1+all_rn[t][k])*(.5*(prices[t+1][k+1] + prices[t+1][k])+all_payments[t+1][k])
        prices[t]= tprices
    return prices


def backward_swaption(exercise_date=5, K=0.062, F=1000):
    swap_prices= backward_swap()
    terminal_val= swap_prices[exercise_date]
    #don't exercise if neg
    terminal_val[terminal_val<0]=0
    prices={exercise_date:terminal_val}
    remain_time= np.arange(exercise_date-1, -1, -1)
    for t in remain_time:
        tprices= np.empty(t+1)
        num_heads= np.arange(t+1)
        for k in num_heads:
            tprices[k]= 1/(1+all_rn[t][k])*(.5*(prices[t+1][k+1] + prices[t+1][k]))
        prices[t]= tprices
    return prices

In [4]:
pp.pprint(backward_swap())

{   0: array([16.3037282]),
    1: array([ 42.78064033, -12.21673656]),
    2: array([ 65.05678889,  13.29592349, -35.29313889]),
    3: array([ 82.2736635 ,  34.60582033, -10.41846252, -52.96768215]),
    4: array([ 93.58997928,  50.85561942,  10.2318731 , -28.40236134,
       -65.16034918]),
    5: array([ 98.2103565 ,  61.20552023,  25.79470303,  -8.10313205,
       -40.56431178, -71.66075966]),
    6: array([ 95.41541922,  64.8621223 ,  35.4246481 ,   7.0537647 ,
       -20.29722971, -46.67265475, -72.11458612]),
    7: array([ 84.59202987,  61.10871876,  38.32339261,  16.210067  ,
        -5.25608584, -26.09879682, -46.34075619, -66.00366651]),
    8: array([ 65.26222881,  49.33572085,  33.77041417,  18.55541673,
         3.68024478, -10.86519544, -25.09062494, -39.00540844,
       -52.61856989]),
    9: array([ 37.109375  ,  29.06976744,  21.15384615,  13.35877863,
         5.68181818,  -1.87969925,  -9.32835821, -16.66666667,
       -23.89705882, -31.02189781])}


In [5]:
pp.pprint(backward_swaption())

{   0: array([15.76314964]),
    1: array([25.2757179 ,  8.14215932]),
    2: array([38.73263739, 14.64967882,  2.67683622]),
    3: array([56.15387212, 25.33959695,  5.71772216,  0.        ]),
    4: array([76.34859997, 41.349916  , 12.16731275,  0.        ,  0.        ]),
    5: array([98.2103565 , 61.20552023, 25.79470303,  0.        ,  0.        ,
        0.        ])}


In [9]:
#q5  
all_rn= get_all_rn(np.arange(T), 0.03, 0, 0.0025)
def count_duplicates(time):
    #count duplicate occurrences in tree recombination
    res= {}
    #generate binary seq
    bin_seq= ["".join(seq) for seq in itertools.product("01", repeat=time)]
    for seq in bin_seq:
        num_heads= seq.count("1")
        res[num_heads]= res.get(num_heads, 0) + 1
    return res

def backward_bond(terminal_val=106, q=0.06, F=100):
    #backward induction for vanilla bond 
    prices= {T:[F]*10, T-1:terminal_val/(1+all_rn[T-1])}
    remain_time= np.arange(T-2, -1, -1)
    for t in remain_time:
        #tprices= number of uniq prices at time t
        tprices= np.empty(t+1)
        #num_heads= 0 to t heads at time t
        num_heads= np.arange(t+1)
        for k in num_heads:
            #multiple payments: add fixed coupon 
            tprices[k]= 1/(1+all_rn[t][k])*(.5*(prices[t+1][k+1] + prices[t+1][k])+F*q)
        prices[t]= tprices
    return prices

def discounts(all_rn, n):
    #compute discounts  
    bin_seq= ["".join(seq) for seq in itertools.product("01", repeat=n)]
    dn= {}
    for idx in range(len(bin_seq)):
        seq= bin_seq[idx]
        res= 1+all_rn[0]
        for t in range(1, n):
            tmp= seq[0:t]
            num_heads= tmp.count("1")
            res= res*(1+all_rn[t][num_heads])
        #track num of heads associated with bin seq
        dn[seq]= (1/res, seq.count("1"))
    return dn

def backward_futures(curr_t=5):
    #backward induction for futures 
    prices= backward_bond()
    dup= count_duplicates(curr_t)
    dup_val= np.array(list(dup.values()))
    total= sum(dup_val)
    #E[P5]
    return np.sum(prices[curr_t]*dup_val)/total

def compute_b0T(all_rn, T):
    #compute time 0 price of ZCB maturing at T
    prices= {T:[1], T-1: 1/(1+all_rn[T-1])}
    remain_time= np.arange(T-2, -1, -1)
    for t in remain_time:
        tprices= np.empty(t+1)
        num_heads= np.arange(t+1)
        for k in num_heads:
            tprices[k]= 1/(1+all_rn[t][k])*(.5*(prices[t+1][k+1] + prices[t+1][k]))
        prices[t]= tprices
    return prices

#3 ways of computing forward price For0,5
def backward_forwards1(curr_t=5):
    #use formula: E(D5P5)/E(D5)
    prices= backward_bond()
    pm= prices[curr_t]
    dm_dict= discounts(all_rn, curr_t)
    total= 0
    bot= 0
    len_dict= len(dm_dict.values())
    for k in dm_dict:
        val, num= dm_dict[k]
        total+= val*pm[num]
        bot+= val
    top = total/len_dict
    bot= bot/len_dict
    return top/bot

def backward_forwards2(curr_t=5):
    #by replication: time 0 of p5 payment/B0,5
    vanilla= backward_bond()
    p5= vanilla[curr_t]
    prices= {curr_t: p5}
    remain_time= np.arange(curr_t-1, -1, -1)
    for t in remain_time:
        tprices= np.empty(t+1)
        num_heads= np.arange(t+1)
        for k in num_heads:
            tprices[k]= 1/(1+all_rn[t][k])*(.5*(prices[t+1][k+1] + prices[t+1][k]))
        prices[t]= tprices
    p0= prices[0]
    return p0/compute_b0T(all_rn, curr_t)[0]

def backward_forwards3(curr_t=5, coupon=6):
    #by replication: V0-For0,5*B0,5-6(B0,1+B0,2+B0,3+B0,4+B0,5)=0
    vanilla= backward_bond()
    p0= vanilla[0]
    for i in range(1,curr_t+1):
        b0= compute_b0T(all_rn, i)[0]
        p0-= b0*coupon
    return p0/compute_b0T(all_rn, curr_t)[0]

In [7]:
pp.pprint(backward_futures())

113.79400972337652


In [11]:
pp.pprint(backward_forwards1())
pp.pprint(backward_forwards2())
pp.pprint(backward_forwards3())

array([113.82420492])
array([113.82420492])
array([113.82420492])


In [12]:
T=11
times= np.arange(T)
all_rn= get_all_rn(np.arange(T), 0.0025, 0.00025, 0.0003)
#q6
def QP():
    prices= {T-1: 100*(1-4*all_rn[T-1])}
    remain_time= np.arange(T-2, -1, -1)
    for t in remain_time:
        tprices= np.empty(t+1)
        num_heads= np.arange(t+1)
        for k in num_heads:
            tprices[k]= (.5*(prices[t+1][k+1] + prices[t+1][k]))
        prices[t]= tprices
    return prices

def backward_eurofutures(K, terminal_time=5):
    QP_res= QP()
    QP510= QP_res[terminal_time]
    call_val= QP510-K
    #take positive part
    call_val[call_val<0]=0
    terminal_val= 2500*call_val
    prices= {terminal_time: terminal_val}
    remain_time= np.arange(terminal_time-1, -1, -1)
    for t in remain_time:
        tprices= np.empty(t+1)
        num_heads= np.arange(t+1)
        for k in num_heads:
            tprices[k]= 1/(1+all_rn[t][k])*(.5*(prices[t+1][k+1] + prices[t+1][k]))
        prices[t]= tprices
    return prices

def all_eurofutures(K=[97.75, 98, 98.25]):
    res= []
    for k in K: 
        prices= backward_eurofutures(k)
        res.append(prices[0])
    return res

In [14]:
pp.pprint(backward_eurofutures(97.75))

{   0: array([685.70485998]),
    1: array([932.11838086, 442.7198634 ]),
    2: array([1215.58026153,  653.22388025,  234.91643772]),
    3: array([1517.56866591,  919.42664241,  390.94046137,   80.58381241]),
    4: array([1820.8121321 , 1221.45777246,  622.82012955,  161.83647047,
          0.        ]),
    5: array([2125., 1525.,  925.,  325.,    0.,    0.])}
